In [1]:
import numpy as np
import pickle
import tensorflow as tf
import sys

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model, load_model

from itertools import permutations 

In [2]:
def train_per_files(eva,path,player,idw,idb):
    white=load_files(path,player,'w',idw)
    black=load_files(path,player,'b',idb)
    total=1
    for i in range(total):
        x,y=merge_data(white,black)
        x,y=shuffle_data(x,y)
        x_train,x_test,y_train,y_test=split_data(x,y)
        eva.fit(x_train, y_train,batch_size=100, epochs=1,validation_data=(x_test,y_test),verbose=0)
        x_train,y_train=flip_data(x_train,y_train)
        x_test,y_test=flip_data(x_test,y_test)
        res=eva.fit(x_train, y_train,batch_size=100, epochs=1,validation_data=(x_test,y_test),verbose=0)
        black = np.roll(black,1)
    return eva,res.history['val_accuracy'][-1]

In [3]:
%run  RandomIndex.ipynb
np.random.seed(1)

In [4]:
eva = tf.keras.Sequential([
      layers.Input(shape=(2, 8, 8, 5)), 
      layers.Conv2D(256, (5,5), activation='relu', padding='same', strides=2),
      layers.Conv2D(64, (3,3), activation='relu', padding='same', strides=2),
      layers.Flatten(),
      layers.Dense(256, activation='sigmoid'),
      layers.Dense(1, activation='sigmoid')])
      #layers.Conv2D(2, kernel_size=(3,3), activation='sigmoid', padding='same')])
eva.compile(optimizer='SGD', loss='binary_crossentropy',metrics=['accuracy'])
eva.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 2, 4, 4, 256)      32256     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 2, 2, 2, 64)       147520    
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 311,361
Trainable params: 311,361
Non-trainable params: 0
_________________________________________________________________


In [5]:
path='pgn/'

In [6]:
with open('pm_b.pkl', 'rb') as infile:
        perm = np.array(pickle.load(infile))

In [ ]:

player='b' #jugador en turno blancas
total=len(perm)
for ciclos in range(8):
    cont=0
    print(f'\nCiclo: {ciclos}')
    for p in perm:
        i=p[0]
        j=p[1]
        
        eva,acc=train_per_files(eva,path,player,i,j)
        sys.stdout.write(f'\r p {cont}/{total}:{i},{j} Accuracy= {acc}           ')
        sys.stdout.flush()
        cont+=1
        if cont%1000==0:
            print(f'\r p {cont}/{total}:{i},{j} Accuracy= {acc} ')
            eva.save('black_nn_'+str(cont//1000)+'.h5')
            rand_state = np.random.get_state()
            name='rs_b_'+str(cont//1000)+'.pkl'
            with open(name, 'wb') as outfile:
                    pickle.dump(rand_state, outfile, pickle.HIGHEST_PROTOCOL)
            print(f'\n{name}')


Ciclo: 0
 p 1000/22500:30,30 Accuracy= 0.8022000193595886            

rs_b_1.pkl
 p 2000/22500:54,34 Accuracy= 0.8119999766349792             

rs_b_2.pkl
 p 3000/22500:33,111 Accuracy= 0.8529999852180481            

rs_b_3.pkl
 p 4000/22500:5,43 Accuracy= 0.8601999878883362              

rs_b_4.pkl
 p 5000/22500:14,127 Accuracy= 0.8802000284194946            

rs_b_5.pkl
 p 6000/22500:3,9 Accuracy= 0.8450000286102295               

rs_b_6.pkl
 p 7000/22500:59,98 Accuracy= 0.868399977684021              

rs_b_7.pkl
 p 8000/22500:8,136 Accuracy= 0.8912000060081482             

rs_b_8.pkl
 p 9000/22500:13,120 Accuracy= 0.8917999863624573            

rs_b_9.pkl
 p 10000/22500:87,119 Accuracy= 0.8820000290870667           

rs_b_10.pkl
 p 11000/22500:123,27 Accuracy= 0.8601999878883362            

rs_b_11.pkl
 p 12000/22500:120,86 Accuracy= 0.8586000204086304            

rs_b_12.pkl
 p 13000/22500:50,84 Accuracy= 0.8550000190734863             

rs_b_13.pkl
 p 14000/22500:119,92 

In [ ]:
eva.save('black_nn.h5')

In [ ]:
new_model = load_model(black_nn.h5')
new_model.summary()

In [ ]:
perm.shape